In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd 
from matplotlib import pyplot
from pandas import to_datetime
from prophet import Prophet
import itertools
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from keras.layers import ConvLSTM2D
from keras.layers import Flatten
from dateutil.relativedelta import relativedelta

Importing plotly failed. Interactive plots will not work.


In [2]:
import keras.backend as K
import tensorflow as tf
# import yfinance as yf
import matplotlib.pyplot as plt
import time
import os
import pickle
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
def prepare_data(timeseries_data, n_features):
    
    '''This function creates multi-dimensional array for 
        time series data'''
    X, y =[],[]
    for i in range(len(timeseries_data)):
        # find the end of this pattern
        end_ix = i + n_features
        # check if we are beyond the sequence
        if end_ix > len(timeseries_data)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def exponential_smoothing(series, alpha, df_name):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    df_name = data.assign(smoothen = result)
    
    return df_name

def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result


def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
    return np.array(X), np.array(y)

In [4]:
data = pd.read_excel('/Users/shubhijain/downloads/Procurement/Raw data/Soybean_oil_May1993_to_May2023.xlsx')

In [6]:
data.head()

,Date,Price,Open,High,low,Volume,Chg%,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,2023-05-17,46.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-05-16,47.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-05-15,49.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-05-12,49.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-05-11,51.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data = data[['Date', 'Price']]

In [11]:
data.head()

,Date,Price,smoothen
1,2023-05-16,47.47,46.363000
2,2023-05-15,49.71,46.697700
3,2023-05-12,49.52,46.979930
4,2023-05-11,51.15,47.396937
5,2023-05-10,52.05,47.862243


In [10]:
data = exponential_smoothing(data.Price, 0.1, "df07")
data = data[1:]

In [18]:
# data = pd.read_excel('/Users/shubhijain/downloads/Soybean_oil_May1993_to_May2023.xlsx')
# data = data[['Date (GMT)',  'Line']]
lstm_df = data[['Date', 'smoothen']]

lstm_df.columns = ['ds', 'y']
lstm_df['ds'] = pd.to_datetime(lstm_df['ds'])

all_dates = pd.DataFrame(pd.date_range(start = '2010-03-06' , end = '2023-05-16', freq='D'), columns = ['Date'])

lstm_df1 = all_dates.merge(lstm_df, left_on = 'Date', right_on= 'ds', how = 'left')

lstm_df1['y'] = lstm_df1['y'].ffill()
lstm_df1['y'] = lstm_df1['y'].bfill()
lstm_df1.drop('ds', axis =1, inplace = True)
lstm_df1.rename(columns = {'Date': 'ds'}, inplace = True)

<ipython-input-18-b596e3add1a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lstm_df['ds'] = pd.to_datetime(lstm_df['ds'])


In [21]:
lstm_df1.sort_values(by = 'ds').reset_index(drop = True)

,ds,y
0,2010-03-06,40.210873
1,2010-03-07,40.210873
2,2010-03-08,40.210873
3,2010-03-09,40.155414
4,2010-03-10,40.090460
...,...,...
4815,2023-05-12,46.979930
4816,2023-05-13,46.979930
4817,2023-05-14,46.979930
4818,2023-05-15,46.697700


In [26]:
months = pd.DataFrame(pd.date_range(start = '2015-03-20' , end = '2023-05-17', freq='M'), columns = ['Month'])
months = pd.DataFrame(months['Month'].astype(str))
list_mths = list(months['Month'])
list_mths_20 = [i[:8] + '20'  for i in list_mths]
lstm_op = pd.DataFrame()

In [27]:
# list_mths_20 = ['2015-03-20']

In [28]:
for train_date in list_mths_20: 
    print("#####################################")
    print(train_date)
    train_dt = pd.to_datetime(train_date)
    print(train_dt)
    dt1 = pd.to_datetime(train_dt,format = '%Y-%m-%d') + relativedelta(months = 3)
    print(dt1)
    if ((dt1.month in (1,3,5,7,8,10,12)) & (dt1.day !=31)): 
        dt1 = pd.to_datetime(dt1,format = '%Y-%m-%d')  + relativedelta(days = 1)
    if (train_dt.month ==2) & (train_dt.day ==28):
            dt1 = pd.to_datetime(train_dt,format = '%Y-%m-%d') + relativedelta(months = 1) + relativedelta(days = 3)
    elif (train_dt.month ==2) & (train_dt.day ==29):
        dt1 = pd.to_datetime(train_dt,format = '%Y-%m-%d') + relativedelta(months = 1) + relativedelta(days = 2)

    print(dt1)
        
    print("##### dt1######")
    print(dt1)
    print("####dt2######")
#     print(dt2)

    str_dt = pd.to_datetime(train_dt,format = '%Y-%m-%d') + relativedelta(days = 1)
    n_days = (dt1-train_dt).days
    all_dts_pred =  pd.DataFrame(pd.date_range(start = str_dt , end = dt1, freq='D'), columns = ['ds'])
    #all_dts_pred =  pd.DataFrame(pd.date_range(start = '2020-04-01' , end = '2020-05-31', freq='D'), columns = ['ds'])
    ref_month = str(train_dt)
    
    lstm_df1_train = lstm_df1[lstm_df1['ds'] <= train_dt]
    print("last value of training set")
    print(lstm_df1_train.tail(1))
    timeseries_data = list(lstm_df1_train['y'])
     
    
    # choose a number of time steps
    n_steps =20
    # split into samples
    X, y = split_sequence(timeseries_data, n_steps)
    
    n_features = 1
    n_seq = 10
    n_steps = 2
    
    X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
#     define model
    model = Sequential()
#     model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mape')
    # fit model
    model.fit(X, y, epochs=500, verbose=0)
    # demonstrate prediction
    x_input = np.array(lstm_df1_train['y'].tail(20))
   
    temp_input=list(x_input)
    lst_output=[]
    i=0
#    n_days = 61
    while(i<n_days):

        if(len(temp_input)>20):
            x_input=np.array(temp_input[1:])
    #         print("{} day input {}".format(i,x_input))
    #         print(x_input)
            x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
    #         print(x_input)
            yhat = model.predict(x_input, verbose=0)
    #         print("{} day output {}".format(i,yhat))
            temp_input.append(yhat[0][0])
            temp_input=temp_input[1:]
    #         print(temp_input)
            lst_output.append(yhat[0][0])
            i=i+1
        else:
            x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
    #         print(yhat[0])
            temp_input.append(yhat[0][0])
            lst_output.append(yhat[0][0])
            i=i+1
   
    print("length of lst output")
    len(lst_output)
    pred_df = all_dts_pred.copy()
    pred_df['pred'] = lst_output
    lstm_pred = pred_df.merge(lstm_df1, on = 'ds', how = 'left')
    lstm_pred['ref_month'] = ref_month
    lstm_pred['start_dt'] = dt1
#     lstm_pred['dt2'] = dt2
    lstm_pred['n_days'] = n_days
    lstm_op = lstm_op.append(lstm_pred)

#####################################
2015-03-20
2015-03-20 00:00:00
2015-06-20 00:00:00
2015-06-20 00:00:00
##### dt1######
2015-06-20 00:00:00
####dt2######
last value of training set
             ds          y
1840 2015-03-20  31.324657
length of lst output
#####################################
2015-04-20
2015-04-20 00:00:00
2015-07-20 00:00:00
2015-07-21 00:00:00
##### dt1######
2015-07-21 00:00:00
####dt2######
last value of training set
             ds          y
1871 2015-04-20  32.242032
length of lst output
#####################################
2015-05-20
2015-05-20 00:00:00
2015-08-20 00:00:00
2015-08-21 00:00:00
##### dt1######
2015-08-21 00:00:00
####dt2######
last value of training set
             ds          y
1901 2015-05-20  32.951578
length of lst output
#####################################
2015-06-20
2015-06-20 00:00:00
2015-09-20 00:00:00
2015-09-20 00:00:00
##### dt1######
2015-09-20 00:00:00
####dt2######
last value of training set
             ds          y
1932

length of lst output
#####################################
2017-11-20
2017-11-20 00:00:00
2018-02-20 00:00:00
2018-02-20 00:00:00
##### dt1######
2018-02-20 00:00:00
####dt2######
last value of training set
             ds          y
2816 2017-11-20  33.991502
length of lst output
#####################################
2017-12-20
2017-12-20 00:00:00
2018-03-20 00:00:00
2018-03-21 00:00:00
##### dt1######
2018-03-21 00:00:00
####dt2######
last value of training set
             ds          y
2846 2017-12-20  33.324985
length of lst output
#####################################
2018-01-20
2018-01-20 00:00:00
2018-04-20 00:00:00
2018-04-20 00:00:00
##### dt1######
2018-04-20 00:00:00
####dt2######
last value of training set
             ds          y
2877 2018-01-20  32.834381
length of lst output
#####################################
2018-02-20
2018-02-20 00:00:00
2018-05-20 00:00:00
2018-05-21 00:00:00
##### dt1######
2018-05-21 00:00:00
####dt2######
last value of training set
          

length of lst output
#####################################
2020-07-20
2020-07-20 00:00:00
2020-10-20 00:00:00
2020-10-21 00:00:00
##### dt1######
2020-10-21 00:00:00
####dt2######
last value of training set
             ds          y
3789 2020-07-20  30.578398
length of lst output
#####################################
2020-08-20
2020-08-20 00:00:00
2020-11-20 00:00:00
2020-11-20 00:00:00
##### dt1######
2020-11-20 00:00:00
####dt2######
last value of training set
             ds          y
3820 2020-08-20  32.811036
length of lst output
#####################################
2020-09-20
2020-09-20 00:00:00
2020-12-20 00:00:00
2020-12-21 00:00:00
##### dt1######
2020-12-21 00:00:00
####dt2######
last value of training set
             ds         y
3851 2020-09-20  33.44456
length of lst output
#####################################
2020-10-20
2020-10-20 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
##### dt1######
2021-01-21 00:00:00
####dt2######
last value of training set
            

length of lst output
#####################################
2023-03-20
2023-03-20 00:00:00
2023-06-20 00:00:00
2023-06-20 00:00:00
##### dt1######
2023-06-20 00:00:00
####dt2######
last value of training set
             ds          y
4762 2023-03-20  54.721488
length of lst output
#####################################
2023-04-20
2023-04-20 00:00:00
2023-07-20 00:00:00
2023-07-21 00:00:00
##### dt1######
2023-07-21 00:00:00
####dt2######
last value of training set
             ds          y
4793 2023-04-20  51.643218
length of lst output


In [31]:
lstm_op.head()

,ds,pred,y,ref_month,start_dt,n_days
0,2015-03-21,30.982880,31.324657,2015-03-20 00:00:00,2015-06-20,92
1,2015-03-22,30.949753,31.324657,2015-03-20 00:00:00,2015-06-20,92
2,2015-03-23,30.822788,31.366285,2015-03-20 00:00:00,2015-06-20,92
3,2015-03-24,30.780226,31.361428,2015-03-20 00:00:00,2015-06-20,92
4,2015-03-25,30.753914,31.360476,2015-03-20 00:00:00,2015-06-20,92


In [30]:
lstm_op['ds'].max()

Timestamp('2023-07-21 00:00:00')

In [32]:
lstm_op.to_excel('conv_lstm_op_m+1_SBO.xlsx', index = False)